In [1]:
# !pip install mediapipe opencv-python
# !pip3 install tensorflowjs
# !pip install protobuf==3.20.*
# !pip uninstall mediapipe
# !pip install mlserver==1.0.1 grpcio==3.20.1

In [32]:
import cv2
import mediapipe as mp
import tensorflow as tf
import csv
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# import tensorflowjs as tfjs
# import tensorflowjs as tfjs

In [33]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [4]:
# cap = cv2.VideoCapture(0)
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#   while cap.isOpened():
#     success, image = cap.read()
#     if not success:
#       print("Unable to record")
#       continue
#     image.flags.writeable = False
#     image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#     results = pose.process(image)

#     image.flags.writeable = True
#     image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

#     cv2.imshow('MediaPipe Pose', cv2.flip(image,1))

#     if cv2.waitKey(5) & 0xFF == ord('q'):
#       break
# cap.release()
# cv2.destroyAllWindows()

In [5]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart.value, axis=1)
#     right = tf.gather(landmarks, right_bodypart.value, axis=1)
#     center = left * 0.5 + right * 0.5
#     return center


# def get_pose_size(landmarks, torso_size_multiplier=2.5):
#     hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)

#   # Shoulders center
#     shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
#                                       BodyPart.RIGHT_SHOULDER)

#   # Torso size as the minimum body size
#     torso_size = tf.linalg.norm(shoulders_center - hips_center)

#   # Pose center
#     pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                      BodyPart.RIGHT_HIP)
#     pose_center_new = tf.expand_dims(pose_center_new, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to
#   # perform substraction
#     pose_center_new = tf.broadcast_to(pose_center_new,
#                                     [tf.size(landmarks) // (17*2), 17, 2])

#   # Dist to pose center
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#   # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#   # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

#     return pose_size


# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
#                                  BodyPart.RIGHT_HIP)
#     pose_center = tf.expand_dims(pose_center, axis=1)
#   # Broadcast the pose center to the same size as the landmark vector to perform
#   # substraction
#     pose_center = tf.broadcast_to(pose_center, 
#                                 [tf.size(landmarks) // (17*2), 17, 2])
#     landmarks = landmarks - pose_center

#   # Scale the landmarks to a constant pose size
#     pose_size = get_pose_size(landmarks)
#     landmarks /= pose_size

#     return landmarks


# def landmarks_to_embedding(landmarks_and_scores):
#     reshaped_inputs = keras.layers.Reshape((33, 3))(landmarks_and_scores)

#   # Normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])

#   # Flatten the normalized landmark coordinates into a vector
#     embedding = keras.layers.Flatten()(landmarks)

#     return embedding

In [6]:
# def get_center_point(landmarks, left_bodypart, right_bodypart):
#     left = tf.gather(landmarks, left_bodypart, 1)
#     right = tf.gather(landmarks, right_bodypart, 1)
#     center = tf.add(tf.mul(left, 0.5), tf.mul(right, 0.5))
#     return center
  

# def get_pose_size(landmarks, torso_size_multiplier = 2.5):
#     hips_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     shoulders_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_SHOULDER,
#       POINTS.RIGHT_SHOULDER
#     )
#     torso_size = tf.norm(tf.sub(shoulders_center, hips_center))
#     pose_center_new = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center_new = tf.expandDims(pose_center_new, 1)

# #     // pose_center_new = tf.broadcastTo(pose_center_new, [1, 17, 2]);
# #     // return: shape(17,2)
#     d = tf.gather(tf.sub(landmarks, pose_center_new), 0, 0)
#     max_dist = tf.max(tf.norm(d, "euclidean", 0))

# #     // normalize scale
#     pose_size = tf.maximum(
#       tf.mul(torso_size, torso_size_multiplier),
#       max_dist
#     )
#     return pose_size

# def normalize_pose_landmarks(landmarks):
#     pose_center = get_center_point(
#       landmarks,
#       POINTS.LEFT_HIP,
#       POINTS.RIGHT_HIP
#     )
#     pose_center = tf.expandDims(pose_center, 1)
# #     // pose_center = tf.broadcastTo(pose_center, [1, 17, 2])
#     landmarks = tf.sub(landmarks, pose_center)

#     pose_size = get_pose_size(landmarks)
#     landmarks = tf.div(landmarks, pose_size)
#     return landmarks

# def landmarks_to_embedding(landmarks):
# #     // normalize landmarks 2D
#     landmarks = normalize_pose_landmarks(tf.expandDims(landmarks, 0))
#     embedding = tf.reshape(landmarks, [1, 99])
# #     // console.log(embedding.array().then((data) => console.log(data)));
#     return embedding
  

In [55]:
# Run this to generate pose coordinate
csv_out_path = 'pose_coordinatenew.csv'
csv_angles_path = 'pose_angles.csv'
with open(csv_out_path, 'w') as csv_out_file, open(csv_angles_path,'w') as anglefile:
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    csv_out_writer1 = csv.writer(anglefile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
    basepath = 'train'
    for entry in os.listdir(basepath):
        folder_path = os.path.join(basepath, entry)
        for img in os.listdir(folder_path):
            with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                image_path = os.path.join(folder_path,img)
                image = cv2.imread(image_path)

                image.flags.writeable = False
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
                result = pose.process(image)
                pose_landmarks = result.pose_landmarks
                output_frame = image.copy()
                
                if pose_landmarks is not None:
                    # Check the number of landmarks and take pose landmarks.
                    assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
                    pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                    frame_height, frame_width = output_frame.shape[:2]
                    pose_landmarks *= np.array([frame_width, frame_height, frame_width])
                    
                    a = pose_landmarks[12]
                    b = pose_landmarks[14]
                    c = pose_landmarks[16]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    relbow = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[11]
                    b = pose_landmarks[13]
                    c = pose_landmarks[15]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    lelbow = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[14]
                    b = pose_landmarks[12]
                    c = pose_landmarks[24]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    lshoulder = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[13]
                    b = pose_landmarks[11]
                    c = pose_landmarks[23]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    rshoulder = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[24]
                    b = pose_landmarks[26]
                    c = pose_landmarks[28]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    rknee = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[23]
                    b = pose_landmarks[25]
                    c = pose_landmarks[27]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    lknee = np.degrees(np.cos(angle))
                    
                    a = pose_landmarks[26]
                    b = (pose_landmarks[24] + pose_landmarks[23])/2
                    c = pose_landmarks[25]
                    e = a-b
                    f = b-c
                    angle = np.dot(f,e)
                    legs = np.degrees(np.cos(angle))
                    
                    
                    csv_out_writer1.writerow([img, entry, relbow, lelbow, lshoulder, rshoulder, rknee, lknee,legs])
                    pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
                    csv_out_writer.writerow([img, entry] + pose_landmarks)
                    
#                     print([img, entry, relbow, lelbow, lshoulder, rshoulder, rknee, lknee,legs])
                    
    
                    
                    
                

In [8]:
# def get_center_point(pose_landmarks):
#     left_hip = pose_landmarks.landmark[23]
#     right_hip = pose_landmarks.landmark[24]
#     hip_center_x = (left_hip.x + right_hip.x)*0.5;
#     hip_center_y = (left_hip.y + right_hip.y)*0.5;
                    
#     left_shoulder = pose_landmarks.landmark[11]
#     right_shoulder = pose_landmarks.landmark[12]
#     shoulder_center_x = (left_shoulder.x + right_shoulder.x)*0.5;
#     shoulder_center_y = (left_shoulder.y + right_shoulder.y)*0.5;
                    
#     torso_center_x = (hip_center_x + shoulder_center_x )*0.5;
#     torso_center_y = (hip_center_y + shoulder_center_y )*0.5;

#     return torso_center_x, torso_center_y

In [9]:
# def normalize(landmarks,pose_center_new):
#     d = tf.gather(landmarks - pose_center_new, 0, axis=0,
#                 name="dist_to_pose_center")
#       # Max dist to pose center
#     max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

#       # Normalize scale
#     pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)

In [10]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
#                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
#                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x - torso_center_x, lmk.y - torso_center_y, lmk.z] for lmk in pose_landmarks.landmark]

#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [11]:

# csv_out_path = 'pose_coordinate.csv'
# with open(csv_out_path, 'w') as csv_out_file:
#     csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
#     basepath = 'train'
#     for entry in os.listdir(basepath):
#         folder_path = os.path.join(basepath, entry)
#         for img in os.listdir(folder_path):
#             with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
#                 image_path = os.path.join(folder_path,img)
#                 image = cv2.imread(image_path)

#                 image.flags.writeable = False
#                 image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
#                 result = pose.process(image)
#                 pose_landmarks = result.pose_landmarks
#                 output_frame = image.copy()
                
#                 if pose_landmarks is not None:
# #                     torso_center_x, torso_center_y = get_center_point(pose_landmarks)
# #                     print(torso_center_x, torso_center_y)
# #                     # Check the number of landmarks and take pose landmarks.
#                     assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
#                     pose_landmarks = [[lmk.x , lmk.y , lmk.z] for lmk in pose_landmarks.landmark]
#                     pose_landmarks = landmarks_to_embedding(pose_landmarks)
#                     # Map pose landmarks from [0, 1] range to absolute coordinates to get
#                     # correct aspect ratio.
#                     frame_height, frame_width = output_frame.shape[:2]
#                     pose_landmarks *= np.array([frame_width, frame_height, frame_width])

#                     # Write pose sample to CSV.
#                     pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(str).tolist()
#                     csv_out_writer.writerow([img, entry] + pose_landmarks)



In [4]:
df = pd.read_csv('pose_coordinatenew.csv',names=[x for x in range(101)])

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,girl1_chair070.jpg,chair,161.93193,91.50034,-113.45271,161.39712,86.31226,-106.83022,161.58342,86.05865,...,151.52425,145.37344,263.04284,35.51275,164.25312,266.42942,134.10185,161.96825,272.50385,2.88960
1,girl1_chair070_flipped.jpg,chair,139.86855,91.82668,-104.83980,143.78042,87.16641,-108.79897,145.48917,87.27382,...,48.08073,150.45469,237.26957,172.43419,138.36648,272.82157,21.53601,134.58295,257.23835,168.02629
2,girl1_chair075.jpg,chair,167.14772,99.77740,-93.39951,166.52445,95.02889,-87.08542,166.49821,94.65935,...,133.62957,146.21243,266.85129,35.65139,168.46964,268.35528,129.69736,168.64370,275.69879,28.96146
3,girl1_chair075_flipped.jpg,chair,136.16109,100.78100,-76.01230,139.12286,96.90815,-79.26770,140.43436,96.86092,...,-41.07904,145.33850,236.36127,128.01884,133.34532,273.72544,-64.76517,132.31215,257.97881,127.25959
4,girl1_chair076.jpg,chair,168.41848,104.34607,-96.49125,168.76974,99.73064,-89.25885,169.08849,99.53190,...,119.36260,146.60322,268.53570,22.31681,169.18559,264.90762,118.56466,172.15129,276.03448,13.66442


In [6]:
df.shape

(2544, 101)

In [7]:
df.columns

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
           dtype='int64', length=101)

In [8]:
df.drop(0,axis=1,inplace=True)

In [9]:
df.iloc[:,1:].values

array([[ 161.93193,   91.50034, -113.45271, ...,  161.96825,  272.50385,
           2.8896 ],
       [ 139.86855,   91.82668, -104.8398 , ...,  134.58295,  257.23835,
         168.02629],
       [ 167.14772,   99.7774 ,  -93.39951, ...,  168.6437 ,  275.69879,
          28.96146],
       ...,
       [ 160.03075,  101.21098, -275.6927 , ...,  169.06106,   76.79547,
          44.96757],
       [ 135.92988,   85.11297, -182.00737, ...,  134.54167,  275.22699,
          46.33296],
       [ 164.71746,   97.26677, -219.01392, ...,  180.18463,  104.69069,
         416.26228]])

In [10]:
x = df.iloc[:,1:].values

In [11]:
lis = [x for x in df[1]]

In [12]:
le = LabelEncoder()

In [13]:
le.fit(df[1])

LabelEncoder()

In [14]:
lis = le.transform(df[1])

In [15]:
df[1].unique()

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior', 'warrior Balanced'], dtype=object)

In [16]:
lis

array([0, 0, 0, ..., 8, 8, 8])

In [17]:
import keras

In [18]:
y = keras.utils.to_categorical(lis,num_classes=9)

In [20]:
y.shape

(2544, 9)

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42)

In [23]:
x_train.shape

(1908, 99)

In [24]:
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [25]:
x.shape

(2544, 99)

In [26]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=x.shape[1]))
model.add(Dropout(.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25600     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 9)                 585       
                                                                 
Total params: 67,337
Trainable params: 67,337
Non-traina

In [27]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=20, min_delta =0.1 ,restore_best_weights=True)

In [28]:
history = model.fit(x_train, y_train, batch_size=25,epochs=1000,callbacks=[callback])

Epoch 1/1000
77/77 [==============================] - 2s 4ms/step - loss: 28.7162 - accuracy: 0.3716
Epoch 2/1000
77/77 [==============================] - 0s 4ms/step - loss: 7.0765 - accuracy: 0.5168
Epoch 3/1000
77/77 [==============================] - 0s 4ms/step - loss: 3.9343 - accuracy: 0.5828
Epoch 4/1000
77/77 [==============================] - 0s 5ms/step - loss: 2.6091 - accuracy: 0.6483
Epoch 5/1000
77/77 [==============================] - 0s 5ms/step - loss: 2.2997 - accuracy: 0.6677
Epoch 6/1000
77/77 [==============================] - 0s 5ms/step - loss: 1.8314 - accuracy: 0.6855
Epoch 7/1000
77/77 [==============================] - 0s 5ms/step - loss: 1.3718 - accuracy: 0.7233
Epoch 8/1000
77/77 [==============================] - 0s 4ms/step - loss: 1.3078 - accuracy: 0.7369
Epoch 9/1000
77/77 [==============================] - 0s 5ms/step - loss: 1.0540 - accuracy: 0.7537
Epoch 10/1000
77/77 [==============================] - 0s 5ms/step - loss: 0.9563 - accuracy: 0.766

In [29]:
model.predict(x_test)

20/20 [==============================] - 0s 2ms/step


array([[8.9437103e-01, 7.0804870e-04, 5.3343328e-04, ..., 3.7694857e-02,
        5.4284845e-02, 4.8834793e-03],
       [1.4605747e-11, 1.6393482e-08, 1.2122202e-11, ..., 1.1202080e-14,
        7.1319104e-14, 1.0000000e+00],
       [3.1048225e-04, 2.5440164e-05, 1.1448844e-04, ..., 9.9343771e-01,
        3.8744516e-03, 1.2433162e-03],
       ...,
       [2.6045620e-06, 6.1236118e-05, 9.9775451e-01, ..., 1.8561281e-08,
        3.9168464e-07, 2.1715204e-03],
       [6.8166492e-12, 2.7499211e-10, 1.1818849e-14, ..., 9.7430134e-14,
        9.9980134e-01, 8.0560444e-13],
       [2.5064632e-02, 1.0041831e-02, 6.6758674e-03, ..., 8.4104604e-01,
        3.4845848e-02, 3.1131707e-02]], dtype=float32)

In [30]:
model.evaluate(x_test,y_test)

20/20 [==============================] - 0s 3ms/step - loss: 0.3189 - accuracy: 0.9481


[0.3189307451248169, 0.948113203048706]

In [31]:
model.get_weights()

[array([[-0.09492233, -0.08835081, -0.06444798, ..., -0.0288044 ,
          0.00386744,  0.10607471],
        [-0.03674482, -0.04983445, -0.13752174, ...,  0.0183366 ,
          0.06757146, -0.11877972],
        [-0.05134505, -0.07668603,  0.13307507, ..., -0.10627155,
          0.10791592, -0.12384898],
        ...,
        [-0.00961717, -0.11519805,  0.14341463, ...,  0.10264196,
         -0.10851911, -0.04857549],
        [ 0.0621338 ,  0.08883914,  0.10518128, ...,  0.01444143,
          0.12773368, -0.09498715],
        [-0.06026208,  0.08084305, -0.0836506 , ...,  0.05884588,
         -0.04812075,  0.04536082]], dtype=float32),
 array([-0.04900849, -0.0220795 ,  0.0637596 , -0.02359999,  0.00812366,
         0.01901092, -0.00142578, -0.02597215,  0.05285602, -0.04182245,
         0.01582214,  0.01590618, -0.04606887, -0.02018164, -0.01638631,
         0.00757577, -0.01756559, -0.04810071, -0.01825904,  0.        ,
        -0.01351967, -0.05332926, -0.03972177, -0.009439  ,  0.037

In [39]:
np.expand_dims(x_test[0],axis=0).shape

(1, 99)

In [40]:
import pickle

In [41]:
pickle.dump(model, open('model_pkl.pkl','wb'))

INFO:tensorflow:Assets written to: ram://eb520d8b-38be-40d3-a58a-d20546ba9f73/assets


In [42]:
# model_pkl = pickle.load(open('model_pkl.pkl','rb'))

In [43]:
model.predict(np.expand_dims(x_test[0],axis=0))

1/1 [==============================] - 0s 53ms/step


array([[9.85777855e-01, 2.97643012e-08, 1.67123271e-05, 5.80652613e-06,
        1.07634679e-10, 3.37739657e-05, 1.02005815e-02, 2.13246234e-03,
        1.83275365e-03]], dtype=float32)

In [44]:
# mapping = dict(zip(le.classes_, le.transform(le.classes_)))

In [36]:
le.inverse_transform([0])

array(['chair'], dtype=object)

In [41]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Unable to record")
            continue
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB )
        results = pose.process(image)
        pose_landmarks = results.pose_landmarks
        output_frame = image.copy()
        pose_name = 'Unidentified'
        if pose_landmarks is not None:
#             torso_center_x, torso_center_y = get_center_point(pose_landmarks)
            vector = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
            frame_height, frame_width = output_frame.shape[:2]

            vector *= np.array([frame_width, frame_height, frame_width])
            vector = vector.flatten()
            vector = np.expand_dims(vector,axis=0)

            prediction = model.predict((vector))

            index = np.argmax(prediction)

            pose_name = le.inverse_transform([index])
#             print("pose name:",pose_name)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, landmark_drawing_spec = mp_drawing_styles.get_default_pose_landmarks_style())

        image = cv2.flip(image,1)

        #Rectangle
        start_point = (0, 0)
        end_point = (300, 100)
        color = (0, 0, 0)
        thickness = -1
        image = cv2.rectangle(image, start_point, end_point, color, thickness)
        #write
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (25, 50)
        fontScale = 1
        color = (255, 255, 255)
        thickness = 2

        image = cv2.putText(image, pose_name[0], org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)
        cv2.imshow('MediaPipe Pose',image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 33ms/step


In [37]:
#EOF

In [47]:
df[1].unique()

array(['chair', 'cobra', 'dog', 'no_pose', 'shoudler_stand', 'traingle',
       'tree', 'warrior', 'warrior Balanced'], dtype=object)

In [48]:
pose_dict={};

In [49]:
for pose in df[1].unique():
    pose_dict[str(le.transform([pose])[0])] = pose

In [50]:
le.transform(['chair'])

array([0])

In [51]:
pose_dict

{'0': 'chair',
 '1': 'cobra',
 '2': 'dog',
 '3': 'no_pose',
 '4': 'shoudler_stand',
 '5': 'traingle',
 '6': 'tree',
 '7': 'warrior',
 '8': 'warrior Balanced'}

In [52]:
import json

In [53]:
with open('pose_labels.txt','w') as f:
    f.write(json.dumps(pose_dict))

In [54]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

print('Model size: %dKB' % (len(tflite_model) / 1024))

with open('pose_classifier.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\sagoi\AppData\Local\Temp\tmp75cfc4r_\assets
Model size: 71KB


In [55]:
# with open('pose_labels.txt', 'w') as f:
#   f.write('\n'.join(df[1]))

In [56]:
# from zipfile import ZipFile

In [57]:
# with ZipFile('model.zip', 'w') as zipObj:
#     zipObj.write('pose_classifier.tflite')
#     zipObj.write('pose_labels.txt')
#     zipObj.close()

In [58]:
# tfjs.converters.save_keras_model(model,'model')

In [59]:
# !zip pose_classifier.zip pose_labels.txt pose_classifier.tflite

In [60]:
# try:
#   from google.colab import files
#   files.download('pose_classifier.zip')
# except:
#   pass

In [61]:
model.predict(x_test)

20/20 [==============================] - 0s 6ms/step


array([[9.8577780e-01, 2.9764356e-08, 1.6712342e-05, ..., 1.0200592e-02,
        2.1324651e-03, 1.8327544e-03],
       [2.7433457e-04, 2.1371872e-05, 6.7735442e-07, ..., 5.2732042e-07,
        1.4193794e-04, 9.9955970e-01],
       [2.1944558e-02, 1.0433935e-05, 4.6506710e-04, ..., 9.6038407e-01,
        9.6011292e-03, 2.3431632e-04],
       ...,
       [6.7266751e-06, 9.2336594e-04, 9.9166542e-01, ..., 2.5574287e-05,
        3.7513910e-05, 6.7353095e-03],
       [6.4231204e-10, 7.8152130e-15, 1.4526008e-08, ..., 2.0807258e-09,
        1.0000000e+00, 4.8667914e-09],
       [1.2928301e-01, 1.0341634e-03, 5.1784734e-03, ..., 8.0304277e-01,
        2.4340337e-02, 7.3612458e-03]], dtype=float32)

In [62]:
model.evaluate(x_test, y_test)

20/20 [==============================] - 0s 5ms/step - loss: 0.2302 - accuracy: 0.9418


[0.23016199469566345, 0.9418238997459412]